<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load" data-toc-modified-id="Load-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Model-1" data-toc-modified-id="Model-1-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model 1</a></span></li></ul></div>

# Load

In [ ]:
import pickle

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd

In [ ]:
filename="data/stats_model_v1"

In [ ]:
infile = open(filename,'rb')
stats = pickle.load(infile)
infile.close()

? should i convert to a dataframe instead of a bulky dict? 

In [ ]:
import numpy as np

# Model 1

- Timer for S/G2
- Sizer for G1/S

Keys of the dict are the parameters of the model, ordered in the following fashion: $(\alpha, \beta_0, \varepsilon, \gamma, transition\_th = M_{G1/S})$. 

For this first run, we have saved: 
- average RB amount and RB concentration over a given cycle
- delta of RB amount of RB concentration over a given cycle

The dimensionless parameters that we can look at are: 
- $ \Pi_1 = \frac{\alpha M_{G1/S} }{\beta_0 + \gamma}$
- $\Pi_2 = \beta/\gamma$

In [ ]:
def Pi1(key): 
    alpha, beta0, epsilon, gamma, M_G1S = key
    return alpha*M_G1S/(beta0 + gamma)

In [ ]:
def Pi2(key):
    alpha, beta0, epsilon, gamma, M_G1S = key
    return beta0/gamma

---

In [ ]:
keys = list(stats.keys())

In [ ]:
pi1 = np.array([Pi1(key) for key in keys])
pi2 = np.array([Pi2(key) for key in keys])

In [ ]:
pairs = list(zip(pi1, pi2))

In [ ]:
unique_pairs = []
for p in pairs:
    if p not in unique_pairs:
        unique_pairs.append(p)

In [ ]:
len(unique_pairs)/len(pairs)

About 20 % are actually unique. 

---

Let us first scatter plot all of these metrics: 

In [ ]:
from itertools import product

import numpy as np


In [ ]:
plt.scatter(pi1, pi2)
plt.xscale('log')
plt.yscale('log')

In [ ]:
shift = .1

min_length = 5

fig, ax = plt.subplots(2, 2, figsize = (10, 10))
keys_stats = list(stats[keys[0]].keys())
for i, j in product(range(2), repeat=2):
    k = i + 2*j
    
    colors = []
    for m in keys:
        crt_ = stats[m][keys_stats[k]]
        
        if len(crt_)<=min_length:
            colors.append(np.nan)
            
        else:
            if keys_stats[k].endswith("delta"):
                val = crt_[-1]/stats[m][keys_stats[k].split("delta")[0]+"avg"][-1]
                title = keys_stats[k] + "/" + keys_stats[k].split("delta")[0]+"avg"
            else:
                val = crt_[-1]
                title = keys_stats[k]
            colors.append(val)
            
    valid = ~np.isnan(np.array(colors))
    p = ax[i,j].scatter(
        pi1[valid]*(1+shift*np.random.randn(len(valid[valid]))), 
        pi2[valid]*(1+shift*np.random.randn(len(valid[valid]))), 
        s=2,
        c=np.log(np.array(colors)[valid]), alpha=.1
    )
    ax[i,j].set_title(title)
    ax[i,j].set_xscale('log')
    ax[i,j].set_yscale('log')
    ax[i,j].set_xlabel("Pi1")
    ax[i,j].set_ylabel("Pi2")
    fig.colorbar(p, ax=ax[i,j])